In [1]:
import ee
import geemap
from ipywidgets import Select, Button, Layout, VBox

# Authenticate and initialize Earth Engine
#ee.Authenticate()
ee.Initialize()


In [2]:
bangalore = ee.FeatureCollection('users/ujavalgandhi/public/bangalore_boundary')
change = ee.FeatureCollection('users/ujavalgandhi/e2e/bangalore_change_gcps')
nochange = ee.FeatureCollection('users/ujavalgandhi/e2e/bangalore_nochange_gcps')
s2 = ee.ImageCollection('COPERNICUS/S2')

geometry = bangalore.geometry()

rgbVis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

In [3]:
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
        qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask) \
        .select("B.*") \
        .copyProperties(image, ["system:time_start"])

# Create dropdown for year selection
yearDropdown = Select(
    options=['2018', '2019', '2020', '2021', '2022'],
    description='Select a year:',
    style={'description_width': 'initial'},
    layout=Layout(width='100%')
)

In [4]:
def updateImage(change):
    selectedYear = yearDropdown.value
    startDate = ee.Date(selectedYear + '-01-01')
    endDate = ee.Date(selectedYear + '-02-01')

    filtered = s2 \
        .filter(ee.Filter.date(startDate, endDate)) \
        .filter(ee.Filter.bounds(geometry)) \
        .map(maskS2clouds)

    image = filtered.median().clip(geometry)
    Map.addLayer(image, rgbVis, 'Image')

# Add the event handler to the year dropdown
yearDropdown.observe(updateImage, 'value')

# Create a vertical box layout and add the year dropdown
box_layout = Layout(display='flex', flex_flow='column', align_items='stretch', width='300px')
widgets = VBox([yearDropdown], layout=box_layout)

In [5]:
# Create the interactive map
Map = geemap.Map()
Map.addLayer(bangalore, {}, 'Bangalore')
Map.addLayer(change, {'color': 'FF0000'}, 'Change')
Map.addLayer(nochange, {'color': '00FF00'}, 'No Change')
Map.centerObject(geometry, 12)

# Display the widgets and map
VBox([widgets, Map])